In [1]:
!pip install --user --upgrade pip
!pip install --user --upgrade kfp

In [2]:
# Hyperparameters
Registry = 'ckwlsgur20'
ImageNAME = 'mnist'
ImageVer = '4.0v'

In [3]:
def print_op(msg):
    return dsl.ContainerOp(
        name='Print',
        image='alpine:3.6',
        command=['echo', msg],
    )

In [4]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler


@dsl.pipeline(
    name="mnist_pipeline",
    description="mnist_pipeline with pv and pvc"
)
def mnist_pipeline(epochs):
    train_test_step = dsl.ContainerOp(
        name="train and test",
        image="{}/{}:{}".format(Registry, ImageNAME, ImageVer),
        arguments=[
            "epochs", epochs,
        ],
        file_outputs={
            'accuracy': '/accuracy.json',
            'mlpipeline-metrics': '/mlpipeline-metrics.json'
        },
        pvolumes={
            "/mnt": dsl.PipelineVolume(pvc="kubeflow")
        },
    )

    baseline = 0.99
    with dsl.Condition(train_test_step.outputs['accuracy'] > baseline):
        print_op('accuracy is higher than {}'.format(baseline))
    with dsl.Condition(train_test_step.outputs['accuracy'] <= baseline):
        print_op('accuracy is lower than {}'.format(baseline))

In [5]:
# Hyperparameters
pipeline_func = mnist_pipeline
ExpName = '4.pytorch-mnist'
Args = {"epochs": 10}

In [6]:
# compile pipeline
from datetime import datetime
runtime = '{}'.format(datetime.now().strftime("%Y%m%d %H%M%S"))
run_name = pipeline_func.__name__ + ' run ' + runtime
kfp.compiler.Compiler().compile(pipeline_func, '{}.zip'.format(ExpName))
# Submit pipeline to kubeflow server directly
client = kfp.Client(host='pipelines-api.kubeflow.svc.cluster.local:8888')
run_result = client.create_run_from_pipeline_func(pipeline_func,
                                                  experiment_name=ExpName,
                                                  run_name=run_name,
                                                  arguments=Args)

/home/jovyan/.local/lib/python3.6/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,
